<a href="https://colab.research.google.com/github/NickVoulg02/Information-Retrieval/blob/main/colbert_test_link3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ColBERTv2: Indexing & Search Notebook
First, we'll import the relevant classes. Note that `Indexer` and `Searcher` are the key actors here. Next, we'll download the necessary dependencies.

In [1]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')


fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2576, done.
remote: Counting objects: 100% (1079/1079), done.
remote: Compressing objects: 100% (337/337), done.
remote: Total 2576 (delta 843), reused 786 (delta 742), pack-reused 1497
Receiving objects: 100% (2576/2576), 2.02 MiB | 11.66 MiB/s, done.
Resolving deltas: 100% (1602/1602), done.


In [2]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/30

In [3]:
import colbert

In [4]:
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

We will use the docs files and the Queries_20 file.

In [5]:
#importing tsv files from personal github repository and creating docs and queries dataset
!git clone https://github.com/NickVoulg02/Information-Retrieval.git
import pandas as pd
from datasets import Dataset
dataset = 'test'
df1 = pd.read_csv("Information-Retrieval/colbert_test/doc_col.tsv", delimiter = '\t', index_col=0)
df2 =  pd.read_csv("Information-Retrieval/colbert_test/queries_20.tsv", delimiter = '\t', index_col=0)
collection = Dataset.from_pandas(df1, preserve_index=True)
query = Dataset.from_pandas(df2, preserve_index=True)
f'Loaded {len(query)} queries and {len(collection):,} passages'

Cloning into 'Information-Retrieval'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 84 (delta 21), reused 15 (delta 15), pack-reused 59
Receiving objects: 100% (84/84), 499.06 KiB | 4.89 MiB/s, done.
Resolving deltas: 100% (30/30), done.


'Loaded 20 queries and 1,209 passages'

In [6]:
print(query[0])
print(collection[0])

{'query': 'WHAT ARE THE EFFECTS OF CALCIUM ON THE PHYSICAL PROPERTIES OF MUCUS FROM CF PATIENTS', 'query_id': 0}
{'doc': 'PSEUDOMONAS AERUGINOSA INFECTION IN CYSTIC FIBROSIS OCCURRENCE OF PRECIPITATING ANTIBODIES AGAINST PSEUDOMONAS AERUGINOSA IN RELATION TO THE CONCENTRATION OF SIXTEEN SERUM PROTEINS AND THE CLINICAL AND RADIOGRAPHICAL STATUS OF THE LUNGS THE SIGNIFICANCE OF PSEUDOMONAS AERUGINOSA INFECTION IN THE RESPIRATORY TRACT OF 9 CYSTIC FIBROSIS PATIENTS HAVE BEEN STUDIED BY MEANS OF IMMUNOELECTROPHORETICAL ANALYSIS OF PATIENTS SERA FOR THE NUMBER OF PRECIPITINS AGAINST PSEUDOMONAS AERUGINOSA AND THE CONCENTRATIONS OF 16 SERUM PROTEINS IN ADDITION THE CLINICAL AND RADIOGRAPHICAL STATUS OF THE LUNGS HAVE BEEN EVALUATED USING 2 SCORING SYSTEMS PRECIPITINS AGAINST PSEUDOMONAS AERUGINOSA WERE DEMONSTRATED IN ALL SERA THE MAXIMUM NUMBER IN ONE SERUM WAS 22 THE CONCENTRATIONS OF 12 OF THE SERUM PROTEINS WERE SIGNIFICANTLY CHANGED COMPARED WITH MATCHED CONTROL PERSONS NOTABLY IGG AND 

## Indexing

For an efficient search, we can pre-compute the ColBERT representation of each passage and index them.

Below, the `Indexer` take a model checkpoint and writes a (compressed) index to disk. We then prepare a `Searcher` for retrieval from this index.

In [7]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens

index_name = f'{dataset}.{nbits}bits'

Now run the `Indexer` on the collection subset. Assuming the use of only one GPU, this cell should take about six minutes to finish running.

In [8]:
checkpoint = 'colbert-ir/colbertv2.0'

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection["doc"], overwrite=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]



[Jan 21, 22:47:37] #> Creating directory /content/experiments/notebook/indexes/test.2bits 


#> Starting...
#> Joined...


In [9]:
indexer.get_index() # You can get the absolute path of the index, if needed.

'/content/experiments/notebook/indexes/test.2bits'

## Search

Having built the index and prepared our `searcher`, we can search for individual query strings.

In [10]:
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=collection["doc"])

[Jan 21, 22:52:07] #> Loading codec...
[Jan 21, 22:52:07] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jan 21, 22:52:08] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jan 21, 22:52:08] #> Loading IVF...
[Jan 21, 22:52:08] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 3398.95it/s]

[Jan 21, 22:52:08] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 100.54it/s]


In [36]:
question = query["query"][1]
print(f"#> {question}")

# Find the top-10 passages for this query
results = searcher.search(question, k=65)
#full length search? check file

passages_ranked = {}
# Print out the top-k retrieved passages
print("Rank\tScore\tId\tPassage")
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"[{passage_rank}] \t{passage_score:.1f} \t{collection['doc_id'][passage_id]} \t{searcher.collection[passage_id]}")
    passages_ranked[passage_id] = passage_rank

#> CAN ONE DISTINGUISH BETWEEN THE EFFECTS OF MUCUS HYPERSECRETION AND INFECTION ON THE SUBMUCOSAL GLANDS OF THE RESPIRATORY TRACT IN CF
Rank	Score	Id	Passage
[1] 	17.8 	437 	PULMONARY ASPECTS OF CYSTIC FIBROSIS PP 324 DEFECTIVE MUCOCILIARY TRANSPORT HAS BEEN IMPLICATED IN THE PATHOGENESIS OF THE LUNG DISEASE IN CYSTIC FIBROSIS BASED ON SEVERAL OBSERVATIONS FIRST PATIENTS WITH THE DISEASE SECRETE MORE MUCUS THAN NORMAL SUBJECTS AS INDICATED BY GLAND HYPERTROPHY AND FREQUENTLY RETAIN IT IN THE LUNG SECOND THE MUCUS IS SAID TO HAVE ALTERED BIOCHEMICAL AND RHEOLOGICAL PROPERTIES HOWEVER ALTHOUGH THE PHYSIOCHEMICAL PROPERTIES OF MUCUS ARE ALTERED IN ALL LUNG DISEASES LITTLE HISTOCHEMICAL OR RHEOLOGICAL DIFFERENCE HAS BEEN FOUND IN MUCUS FROM PATIENTS WITH CF AND OTHER HYPERSECRETORY CONDITIONS SUCH AS CHRONIC BRONCHITIS AND BRONCHIECTASIS THIRD A FACTOR IN THE SERUM OF CF PATIENTS INHIBITS NORMAL CILIARY ACTIVITY IN EXPLAIN GLANDS OF CILIATED EPITHELIUM AND INFECTION WHICH DAMAGES HUMAN CI

In [50]:
f = open("Information-Retrieval/Relevant_20", "r")
file = f.read().splitlines()
relevant_docs = []
for line in file:
    line=line.split()
    list = []
    for num in line:
        num = int(num)
        list.append('doc'+str(num))
    relevant_docs.append(list)                  # relevant_docs list includes every line from Relevant_20

{427: 1, 734: 2, 571: 3, 574: 4, 148: 5, 485: 6, 1145: 7, 169: 8, 1099: 9, 491: 10, 431: 11, 785: 12, 946: 13, 484: 14, 6: 15, 259: 16, 441: 17, 891: 18, 957: 19, 1142: 20, 639: 21, 1207: 22, 516: 23, 470: 24, 487: 25, 325: 26, 373: 27, 134: 28, 770: 29, 1156: 30, 965: 31, 1147: 32, 429: 33, 1143: 34, 933: 35, 735: 36, 445: 37, 995: 38, 1117: 39, 713: 40, 613: 41, 952: 42, 840: 43, 570: 44, 197: 45, 186: 46, 1139: 47, 768: 48, 94: 49, 578: 50, 289: 51, 46: 52, 1177: 53, 254: 54, 756: 55, 173: 56, 488: 57, 1141: 58, 964: 59, 317: 60, 437: 61, 1095: 62, 934: 63, 1053: 64}


In [53]:
#avg_pr_list = []
i=0;
if i == 0:
    precision_at_k = []
    recall_at_k = []
    true_positives = 0

    for key, value in passages_ranked.items():
      doc = "doc"+str(collection['doc_id'][key])
      if(doc in relevant_docs[i]):
        print(doc)
        true_positives+=1
        print(value)
        precision_at_k.append(true_positives/value)
        recall_at_k.append(true_positives/len(relevant_docs[i]))

    average_precision = 0
    for x in range(len(precision_at_k)):
      value = precision_at_k[x]*recall_at_k[x]
      average_precision += value

    #print(average_precision)
    average_precision = average_precision/len(relevant_docs[i])
    print(average_precision)
    #avg_pr_list.append(average_precision)

#mean_average_precision = sum(avg_pr_list)/20
#print(mean_average_precision)

## error error

doc151
5
doc505
10
doc441
11
doc533
23
doc1185
30
doc1175
32
doc439
33
doc502
57
0.005778393449671747
